In [1]:
import pandas as pd
from datetime import datetime
import jdatetime
import re
import tqdm
from tqdm import tqdm
from urlextract import URLExtract
import emojis
import hazm
from hazm import *


In [2]:
df = pd.read_json(r'data/result.json')
df.head()

,name,type,id,messages
0,آمبیدو؛,public_channel,1677493865,"{'id': 1, 'type': 'service', 'date': '2023-03-..."
1,آمبیدو؛,public_channel,1677493865,"{'id': 3, 'type': 'service', 'date': '2023-03-..."
2,آمبیدو؛,public_channel,1677493865,"{'id': 4, 'type': 'message', 'date': '2023-03-..."
3,آمبیدو؛,public_channel,1677493865,"{'id': 7, 'type': 'message', 'date': '2023-03-..."
4,آمبیدو؛,public_channel,1677493865,"{'id': 8, 'type': 'message', 'date': '2023-03-..."


In [3]:
df['messages'][0]

{'id': 1,
 'type': 'service',
 'date': '2023-03-09T01:20:56',
 'date_unixtime': '1678312256',
 'actor': 'آمبیدو؛',
 'actor_id': 'channel1677493865',
 'action': 'create_channel',
 'title': 'به رنگ مینا',
 'text': '',
 'text_entities': []}

In [4]:
expanded_message = pd.json_normalize(df['messages'])
expanded_message.head()

,id,type,date,date_unixtime,actor,actor_id,action,title,text,text_entities,...,forwarded_from,file_name,sticker_emoji,author,performer,reply_to_peer_id,poll.question,poll.closed,poll.total_voters,poll.answers
0,1,service,2023-03-09T01:20:56,1678312256,آمبیدو؛,channel1677493865,create_channel,به رنگ مینا,,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,service,2023-03-09T01:36:16,1678313176,آمبیدو؛,channel1677493865,edit_group_photo,NaN,,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,message,2023-03-09T01:37:38,1678313258,NaN,NaN,NaN,NaN,امروز؛ سحرگاه ۱۸ام اسفندماه، فرزند دومم متولد شد.,"[{'type': 'plain', 'text': 'امروز؛ سحرگاه ۱۸ام...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,message,2023-03-09T02:11:02,1678315262,NaN,NaN,NaN,NaN,من کیستم؟\n\nمبر ز موی سپیدم گمان به عمر دراز\...,"[{'type': 'plain', 'text': 'من کیستم؟ مبر ز م...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,message,2023-03-09T02:20:06,1678315806,NaN,NaN,NaN,NaN,[شاید به دردتان خورد:\n\nبداهه یا حرف‌های روزم...,"[{'type': 'plain', 'text': 'شاید به دردتان خور...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
selected_columns = ['id', 'type', 'date','text']  
selected_df = expanded_message[selected_columns]
selected_df.rename(columns={'type': 'dict_type','id':'dict_id'}, inplace=True)
data = pd.concat([df.drop(columns=['messages']), selected_df], axis=1)

C:\Users\Mostafa\AppData\Local\Temp\ipykernel_2800\2967748843.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df.rename(columns={'type': 'dict_type','id':'dict_id'}, inplace=True)


In [6]:
data.head(6)

,name,type,id,dict_id,dict_type,date,text
0,آمبیدو؛,public_channel,1677493865,1,service,2023-03-09T01:20:56,
1,آمبیدو؛,public_channel,1677493865,3,service,2023-03-09T01:36:16,
2,آمبیدو؛,public_channel,1677493865,4,message,2023-03-09T01:37:38,امروز؛ سحرگاه ۱۸ام اسفندماه، فرزند دومم متولد شد.
3,آمبیدو؛,public_channel,1677493865,7,message,2023-03-09T02:11:02,من کیستم؟\n\nمبر ز موی سپیدم گمان به عمر دراز\...
4,آمبیدو؛,public_channel,1677493865,8,message,2023-03-09T02:20:06,[شاید به دردتان خورد:\n\nبداهه یا حرف‌های روزم...
5,آمبیدو؛,public_channel,1677493865,9,message,2023-03-09T16:00:14,سلام بر راه، که جدایی و وصال همگان از اوست!


In [7]:
print(data['name'].value_counts())
print("-------")
print(data['type'].value_counts())
print("-------")
print(data['dict_type'].value_counts())


آمبیدو؛    4465
Name: name, dtype: int64
-------
public_channel    4465
Name: type, dtype: int64
-------
message    4460
service       5
Name: dict_type, dtype: int64


In [8]:
data = data[data['dict_type']=='message']

### The number of text message in channel is 4458

In [9]:
data.drop(['name','type','dict_type','id'],axis=1,inplace=True)

In [10]:
data.head(2)

,dict_id,date,text
2,4,2023-03-09T01:37:38,امروز؛ سحرگاه ۱۸ام اسفندماه، فرزند دومم متولد شد.
3,7,2023-03-09T02:11:02,من کیستم؟\n\nمبر ز موی سپیدم گمان به عمر دراز\...


In [11]:
data.reset_index(inplace=True,drop=True)

In [12]:
def expand_timestamp(ts):
    dt = datetime.fromisoformat(ts)
    jdt = jdatetime.datetime.fromgregorian(datetime=dt)
    
    return pd.Series({
        'persian_year': jdt.year,
        'persian_month': jdt.month,
        'persian_day': jdt.day,
        'hour': dt.hour,
        'day_of_week': dt.strftime('%A')
        # 'year': dt.year,
        # 'month': dt.month,
        # 'day': dt.day,
        
       
        
    })

expanded_date = data['date'].apply(expand_timestamp)

data = pd.concat([data, expanded_date], axis=1)

In [13]:
data.head(2)

,dict_id,date,text,persian_year,persian_month,persian_day,hour,day_of_week
0,4,2023-03-09T01:37:38,امروز؛ سحرگاه ۱۸ام اسفندماه، فرزند دومم متولد شد.,1401,12,18,1,Thursday
1,7,2023-03-09T02:11:02,من کیستم؟\n\nمبر ز موی سپیدم گمان به عمر دراز\...,1401,12,18,2,Thursday


In [14]:
def _multiple_replace(mapping, text):
    pattern = "|".join(map(re.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))

def convert_fa_numbers(input_str):
    mapping = {
        '۰': '0',
        '۱': '1',
        '۲': '2',
        '۳': '3',
        '۴': '4',
        '۵': '5',
        '۶': '6',
        '۷': '7',
        '۸': '8',
        '۹': '9',
        '.': '.',
    }
    return _multiple_replace(mapping, input_str)


def convert_ar_characters(input_str):
    mapping = {
        'ك': 'ک',
        'ى': 'ی',
        'ي': 'ی',
        'ئ':'ی',
        'إ':'ا',
        'أ':'ا',
        'ة':'ه',
        'ؤ':'و'
    }
    return _multiple_replace(mapping, input_str)

specific_words = ['type','text','bold','italic','link','metion','spoiler','quote','underline','href','to','text_link']
def preprocess(text):
    extractor = URLExtract()
    for url in extractor.gen_urls(text):
        text = text.replace(url,'')

    
    emj = emojis.get(text)
    for i in emj:
        if i in text:
            text = text.replace(i,'')
    text = convert_fa_numbers(text)
    text = convert_ar_characters(text)
    text = re.sub(r"\{'type': 'hashtag', 'text': '[^']+'\}", '', text)
    text = re.sub(r"\{'type': 'text_link', 'text': '[^']+'\}", '', text)
    text = re.sub(r"\{'type': 'text_link', 'text': '[^']+'\}, 'href': '[^']+'\}", '', text)
    text = re.sub(r'\b(?:' + '|'.join(specific_words) + r')\b', '', text)
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r"(:\s?\)|:-\)|\(\s?:|\(-:|:\'\)|:\s?D|8-\)|:\s?\||;\s?\)|:-\*|:-\||:-\(|:\s?P|:-P|:-p|:-b|:-O|:-o|:-0|:-\@|:\$|:-\^|:-&|:-\*|:-\+|:-\~|:-\`|:-\>|:-\<|:-\}|:-\{|\[:\s?\]|\[:\s?\]|:\s?\]|:\s?\[|:\s?\}|:\s?\{)",'<smiley>',text)
    text = text.lower() 
    text = text.strip()
    text = re.sub(r'[<>#.:()"\'!?؟،,@$%^&*_+\[\]/]', ' ', text)
    text = re.sub(r'\\[nu]|\\u200c|\n', '', text)
    text = re.sub(r'200c|\\U[0-9a-fA-F]+', '', text)
    text = re.sub(r'\u200c', '', text)
    text = re.sub(r'\\[uU][0-9a-fA-F]+', '', text)
    text = re.sub(r'[\s]{2,}', ' ', text)
    text = re.sub(r'(\w)\1{2,}', r'\1',text)
    if re.search(r'[\u0600-\u06FF]', text):
        return(text)
    else:
        return(text)
    

data['text'] = data['text'].astype(str)
tqdm.pandas()

data['PP_text'] = data['text'].progress_apply(preprocess)

100%|██████████| 4460/4460 [01:54<00:00, 38.95it/s]


In [15]:
data.head(10)

,dict_id,date,text,persian_year,persian_month,persian_day,hour,day_of_week,PP_text
0,4,2023-03-09T01:37:38,امروز؛ سحرگاه ۱۸ام اسفندماه، فرزند دومم متولد شد.,1401,12,18,1,Thursday,امروز؛ سحرگاه 18ام اسفندماه فرزند دومم متولد شد
1,7,2023-03-09T02:11:02,من کیستم؟\n\nمبر ز موی سپیدم گمان به عمر دراز\...,1401,12,18,2,Thursday,من کیستم مبر ز موی سپیدم گمان به عمر درازجوان ...
2,8,2023-03-09T02:20:06,['شاید به دردتان خورد:\n\nبداهه یا حرف\u200cها...,1401,12,18,2,Thursday,شاید به دردتان خورد بداهه یا حرفهای روزمره ຯپ...
3,9,2023-03-09T16:00:14,سلام بر راه، که جدایی و وصال همگان از اوست!,1401,12,18,16,Thursday,سلام بر راه که جدایی و وصال همگان از اوست
4,10,2023-03-09T17:54:51,این دفعه، زمان بیشتری رو توی تاریکی می‌گذرونیم.,1401,12,18,17,Thursday,این دفعه زمان بیشتری رو توی تاریکی میگذرونیم
5,11,2023-03-09T18:50:35,ساقی به نور باده، بر افروز جان ما!,1401,12,18,18,Thursday,ساقی به نور باده بر افروز جان ما
6,15,2023-03-09T23:41:54,زبون چراغی راننده‌ها توی جاده رو دوست دارم. وق...,1401,12,18,23,Thursday,زبون چراغی رانندهها توی جاده رو دوست دارم وقتی...
7,20,2023-03-10T04:40:01,['چه کسی گفته است فقط در دریا آسمان با زمین پی...,1401,12,19,4,Friday,چه کسی گفته است فقط در دریا آسمان با زمین پیو...
8,22,2023-03-10T09:49:00,صبح می‌خواستم‌ همچین عکسی از طلوع بگیرم. هوا ا...,1401,12,19,9,Friday,صبح میخواستم همچین عکسی از طلوع بگیرم هوا ابری...
9,23,2023-03-10T12:27:47,بعد از پشت سر گذاشتن کوه‌های خشک و قدیمی، بعد ...,1401,12,19,12,Friday,بعد از پشت سر گذاشتن کوههای خشک و قدیمی بعد از...


In [16]:
data["length"] = data["PP_text"].str.len()
tokenizer = WordTokenizer()
data["num_tokens"] = data["PP_text"].apply(lambda text: len(tokenizer.tokenize(text)))

In [20]:
def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text


normalizer= Normalizer()
lemma = Lemmatizer()
stemming=Stemmer()
stopwords = set(stopwords_list())

def normalize_text(text):
    return normalizer.normalize(text)

def stem_text(text):
    words = word_tokenize(text)
    stemmed_words = [stemming.stem(word) for word in words if word not in stopwords]
    return ' '.join(stemmed_words)

def lemmatize_text(text):
    words = word_tokenize(text)
    lemmatized_words = [lemma.lemmatize(word).split('#')[0] for word in words if word not in stopwords]
    return ' '.join(lemmatized_words)

data['PP_text'] = data['PP_text'].apply(normalize_text)

data['lemma_text'] = data['PP_text'].apply(remove_punctuation)
data['lemma_text'] = data['lemma_text'].apply(lemmatize_text)

data['stem_text'] = data['PP_text'].apply(remove_punctuation)
data['stem_text'] = data['stem_text'].apply(stem_text)



In [21]:
data.head()

,dict_id,date,text,persian_year,persian_month,persian_day,hour,day_of_week,PP_text,length,num_tokens,lemma_text,stem_text
0,4,2023-03-09T01:37:38,امروز؛ سحرگاه ۱۸ام اسفندماه، فرزند دومم متولد شد.,1401,12,18,1,Thursday,امروز؛ سحرگاه ۱۸ام اسفندماه فرزند دومم متولد شد,48,10,امروز سحرگاه ۱۸ ام اسفندماه فرزند دوم متولد,امروز سحرگاه ۱۸ ا اسفندماه فرزند دوم متولد
1,7,2023-03-09T02:11:02,من کیستم؟\n\nمبر ز موی سپیدم گمان به عمر دراز\...,1401,12,18,2,Thursday,من کیستم مبر ز موی سپیدم گمان به عمر درازجوان ...,189,36,کیستم مبر ز مو سپید گمان عمر درازجوان ز حادث پ...,کیس مبر ز مو سپید گ عمر درازجو ز حادث پیر میشو...
2,8,2023-03-09T02:20:06,['شاید به دردتان خورد:\n\nبداهه یا حرف\u200cها...,1401,12,18,2,Thursday,شاید به دردتان خورد بداهه یا حرفهای روزمره ຯپ...,298,54,درد خورد بداهه حرف روزمره ຯپسرام هیونگی گرگ এل...,درد خورد بداهه حرف روزمره ຯپسرا هیونگ گرگ এلحظ...
3,9,2023-03-09T16:00:14,سلام بر راه، که جدایی و وصال همگان از اوست!,1401,12,18,16,Thursday,سلام بر راه که جدایی و وصال همگان از اوست,42,10,سلام جدایی وصال همگان اوست,سلا جدا وصال همگ اوس
4,10,2023-03-09T17:54:51,این دفعه، زمان بیشتری رو توی تاریکی می‌گذرونیم.,1401,12,18,17,Thursday,این دفعه زمان بیشتری رو توی تاریکی میگذرونیم,45,8,دفعه زمان تو تاریکی میگذرونیم,دفعه ز تو تاریک میگذرون


In [22]:
data.to_json('data/PP-data.json')